Name: **Luong Nguyen**  
Student ID: **1504210**  

# Introduction to Deep Learning 

## Session07: word-level text generation
____
Based on example code from Chollet's book chapter [8.1](https://github.com/fchollet/deep-learning-with-python-notebooks/blob/master/8.1-text-generation-with-lstm.ipynb)

## Import the packages needed for this assignment

In [2]:
# install keras on google colab
!pip install -q keras
# is GPU working?
import tensorflow as tf
tf.test.gpu_device_name()

'/device:GPU:0'

In [3]:
# import packages
import numpy as np
import keras
from keras import models, layers
from keras.utils.np_utils import to_categorical

Using TensorFlow backend.


## Prepare data

### Load data

I downloaded 10 Metropolia Bsc theses written in English related to Web programming from Theseus, converted them to .txt, concat them into 1 .txt file.

In [4]:
path = keras.utils.get_file(
    'text.txt',
    origin='http://users.metropolia.fi/~luongn/text.txt')
text = open(path).read().lower()
print('Corpus length:', len(text))

Corpus length: 656120


A look at `text`

In [44]:
text[:100]

'befekadu mezgebu temere\n\nresponsive web application using bootstrap\nand foundation\ncomparing bootstr'

### Preprocess data

Replace non-alphanumeric characters with whitespaces, then `split()` to get `words` array.

In [5]:
import re
text = re.sub(r'\W+', ' ', text)

words = text.split()
words[:10]

['befekadu',
 'mezgebu',
 'temere',
 'responsive',
 'web',
 'application',
 'using',
 'bootstrap',
 'and',
 'foundation']

In [6]:
# Length of extracted word sequences
maxlen = 10

# We sample a new sequence every `step` word
step = 3

# This holds our extracted sequences
sentences = []

# This holds the targets (the follow-up words)
next_words = []

for i in range(0, len(words) - maxlen, step):
    sentences.append(words[i: i + maxlen])
    next_words.append(words[i + maxlen])
print('Number of sequences:', len(sentences))


# List of unique words in the corpus
unique_words = sorted(list(set(words)))
print('Unique words:', len(unique_words))
# Dictionary mapping unique words to their index in `unique_words`
word_indices = dict((word, unique_words.index(word)) for word in unique_words)

# Next, one-hot encode the words into binary arrays.
print('Vectorization...')
x = np.zeros((len(sentences), maxlen, len(unique_words)), dtype=np.bool)
y = np.zeros((len(sentences), len(unique_words)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, word in enumerate(sentence):
        x[i, t, word_indices[word]] = 1
    y[i, word_indices[next_words[i]]] = 1
    
print("x shape: %s" % (str(x.shape)))
print("y shape: %s" % (str(y.shape)))

Number of sequences: 35451
Unique words: 6679
Vectorization...
x shape: (35451, 10, 6679)
y shape: (35451, 6679)


## Build network

In [7]:
model = models.Sequential()
model.add(layers.LSTM(128, input_shape=(maxlen, len(unique_words))))
model.add(layers.Dense(len(unique_words), activation='softmax'))
optimizer = keras.optimizers.RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 128)               3485696   
_________________________________________________________________
dense_1 (Dense)              (None, 6679)              861591    
Total params: 4,347,287
Trainable params: 4,347,287
Non-trainable params: 0
_________________________________________________________________


## Train model and sample

Reweight the original probability distribution coming out of the model, and draw a word index from it 

In [0]:
def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

Repeatedly train and generate text

In [10]:
import random
import sys

for epoch in range(1, 40):
    print('epoch', epoch)
    # Fit the model for 1 epoch on the available training data
    model.fit(x, y,
              batch_size=128,
              epochs=1)

    # Select a text seed at random
    start_index = random.randint(0, len(words) - maxlen - 1)
    generated_text = words[start_index: start_index + maxlen]
    generated_text_str = ' '.join(generated_text)
    print('--- Generating with seed: "' + generated_text_str + '"')

    for temperature in [0.2, 0.5, 1.0, 1.2]:
        print('------ temperature:', temperature)
        sys.stdout.write(generated_text_str + " ")

        # We generate 40 words
        for i in range(40):
            sampled = np.zeros((1, maxlen, len(unique_words)))
            for t, word in enumerate(generated_text):
                sampled[0, t, word_indices[word]] = 1.

            preds = model.predict(sampled, verbose=0)[0]
            next_index = sample(preds, temperature)
            next_word = words[next_index]

            generated_text.append(next_word)
            generated_text = generated_text[1:]

            sys.stdout.write(next_word + " ")
            sys.stdout.flush()
        print()

epoch 1
Epoch 1/1
35451/35451 [==============================] - 22s 607us/step - loss: 4.0301
--- Generating with seed: "this section is handled by the dashboard component which is"
------ temperature: 0.2
this section is handled by the dashboard component which is language in external speed 1 paper frontend simplifies external got including 1 paper frontend simplifies external that external 1 is the that website 1 paper 1 1 paper plugins external 1 paper 1 popular frontend got external simplifies external got 
------ temperature: 0.5
this section is handled by the dashboard component which is 1 paper simplifies in external that 1 speed 1 paper resource pagespeed twitter frontend 1 paper mobile according the rules plugins plugins they they they website pagespeed is external paper external performing components syntactically several external above performing 1 external 
------ temperature: 1.0
this section is handled by the dashboard component which is project 1 usage 1 external plugin

the articles figure project better to external makes web 1 foundation device score 
------ temperature: 1.2
this section is handled by the dashboard component which is tv 2 using websites a for installed the optimizilla project the html5 according the of bootstrap bootstrap 4 was vital javascript common search paper were currently the on years foundation according the search the and score and including templates resource 
epoch 2
Epoch 1/1
21632/35451 [=================>............] - ETA: 8s - loss: 3.5326

35451/35451 [==============================] - 21s 603us/step - loss: 3.7181
--- Generating with seed: "ry application has its own requirements and challenges and therefore"
------ temperature: 0.2
ry application has its own requirements and challenges and therefore web about syntactically 1 syntactically external websites website it tv research the desktop pagespeed pagespeed mobile mobile the of external mobile semantic pagespeed following external score and external results is web the that syntactically external more syntactically syntactically above syntactically 
------ temperature: 0.5
ry application has its own requirements and challenges and therefore bootstrap syntactically 1 that of external that they they components syntactically in speed 1 popular they they following external that external and better including the was the internally plugins in external semantic figure was term external external class class a 
------ temperature: 1.0
ry application has its own requirements a

external file and websites 
------ temperature: 1.2
ry application has its own requirements and challenges and therefore befekadu class class class 4 and research foundation of 30 source the was the that speed external the that components 2 35 better structure the before mobile website pagespeed of 808 the in according and in mobile of means abstract 
epoch 3
Epoch 1/1
25984/35451 [====================>.........] - ETA: 5s - loss: 3.3261

35451/35451 [==============================] - 21s 604us/step - loss: 3.4205
--- Generating with seed: "for the foundation website image a screenshot of sample images"
------ temperature: 0.2
for the foundation website image a screenshot of sample images syntactically templates templates they they they they they they components is web user and addition in external got syntactically of external got the easiness the is like web mobile including external a and was is the is including web user 
------ temperature: 0.5
for the foundation website image a screenshot of sample images class class in than is external plugins external 1 external 1 it 1 paper external including external and including got including easiness the constant folder performing plugins is the in including because external syntactically external in and and external got 
------ temperature: 1.0
for the foundation website image a screenshot of sample images that frameworks class pagespeed about simplifies web simplifies exte

...

35451/35451 [==============================] - 21s 605us/step - loss: 1.8983
--- Generating with seed: "two frontend frameworks both frameworks are open source html css"
------ temperature: 0.2
two frontend frameworks both frameworks are open source html css syntactically in and the syntax less web user external templates issues 2 popular figure befekadu better is is external popular was several where external user tv they they they components external research is the where website website 1 navigation foundation 
------ temperature: 0.5
two frontend frameworks both frameworks are open source html css 1 paper was the internally in above paper semantic tv saved syntactically foundation research using see class project on external offers using and a foundation components multitude frameworks they the and the for to 2 get high pagespeed high it 
------ temperature: 1.0
two frontend frameworks both frameworks are open source html css size for for to two second tools plugins chrome 1 millis

will a developers high class a open 
------ temperature: 1.2
two frontend frameworks both frameworks are open source html css quality 2017 and 3 was structure building due according of the 393 background have in less is fix website support folder syntax css last colours around two as support and com a the the the the improving plugins and coding 
epoch 38
Epoch 1/1
26112/35451 [=====================>........] - ETA: 5s - loss: 1.8011

35451/35451 [==============================] - 21s 603us/step - loss: 1.8832
--- Generating with seed: "e2 80 94and revenue accessed 28 may 2017 19 automation"
------ temperature: 0.2
e2 80 94and revenue accessed 28 may 2017 19 automation class complicated complicated speed 1 and were websites user and region web two 1 addition a a for in web video syntactically external external rules figure web and trend term in because external external score class speed above the two 
------ temperature: 0.5
e2 80 94and revenue accessed 28 may 2017 19 automation to to external a web semantic the figure as external term and one 1 figure figure term can a results plugins overflow syntactically the in code 1 2 popular websites and of easiness 1 by in external 38 syntactically as 
------ temperature: 1.0
e2 80 94and revenue accessed 28 may 2017 19 automation are support and however the of install the to another means browser speed bootstrap of the nested to language web moreover relative minified backg

frameworks web both common internally of speed class class source at browser 17 the user it local 8 the dozens less 11 syntactically syntactically syntactically syntactically testing font 8 out 
epoch 39
Epoch 1/1
35451/35451 [==============================] - 21s 606us/step - loss: 1.8880
--- Generating with seed: "1 on the next page shows a comparison between angularjs"
------ temperature: 0.2
1 on the next page shows a comparison between angularjs that websites the than frontend 1 that 1 external they they external 6 figure that of and class external where they they they they components components that it tv before research external they 

they a for research for website mobile 
------ temperature: 0.5
1 on the next page shows a comparison between angularjs rules the latest engineering following minified the minified web in locator less research templates they they they they website external locator user the to than project file frontend rules external foundation syntactically syntactically 2 language than websites to it website 
------ temperature: 1.0
1 on the next page shows a comparison between angularjs insights bootstrap package 9 code 10 4 and using that like window resource a the tablets furthermore built 3 were 1 in for overview plugins 29 different external are was the activity online and moreover and resource class is started 
------ temperature: 1.2
1 on the next page shows a comparison between angularjs the illustrated bootstrap syntactically translates of foundation css user the to in web 14 provided will header 100 results the plugins plugins system 14 website 4 of the to around 5 is external were was foun

**Generated text after 40 epochs**  
--- Generating with seed: "1 on the next page shows a comparison between angularjs"  
------ temperature: 0.2  
1 on the next page shows a comparison between angularjs that websites the than frontend 1 that 1 external they they external 6 figure that of and class external where they they they they components components that it tv before research external they they a for research for website mobile   
------ temperature: 0.5  
1 on the next page shows a comparison between angularjs rules the latest engineering following minified the minified web in locator less research templates they they they they website external locator user the to than project file frontend rules external foundation syntactically syntactically 2 language than websites to it website   
------ temperature: 1.0  
1 on the next page shows a comparison between angularjs insights bootstrap package 9 code 10 4 and using that like window resource a the tablets furthermore built 3 were 1 in for overview plugins 29 different external are was the activity online and moreover and resource class is started   
------ temperature: 1.2  
1 on the next page shows a comparison between angularjs the illustrated bootstrap syntactically translates of foundation css user the to in web 14 provided will header 100 results the plugins plugins system 14 website 4 of the to around 5 is external were was founded optimization full stylesheets are   